# <center>**CAPTCHA Recognition**</center>
CAPTCHAs may be referred to those infuriating images containing the text that needs to be typed in before a person can access a particular website. The full form of CAPTCHA is “Completely Automated Public Turing test to tell Computers and Humans Apart” and as the name suggests it is a way to avert the computer to fill out the form on its own, automatically. However using the concept of deep learning and computer vision, the very purpose of the CAPTCHAs can be defeated. This test can be passed automatically with the help of Convolutional Neural networks(CNN).

In [ ]:
# Import the required Package
import os
import glob
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from PIL import ImageFile
from torch import nn
torch.cuda.empty_cache()
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import metrics

import engine
from model import CaptchaModel
import config
import dataset


## **Helper Function**

In [ ]:
# this function is useful to remove the duplicate charecter that generate by model 

def remove_duplicates(x):
    if len(x) < 2:
        return x
    fin = ""
    for j in x:
        if fin == "":
            fin = j
        else:
            if j == fin[-1]:
                continue
            else:
                fin = fin + j
    return fin

In [ ]:
# this function is useful to decode the prediction 
# after getting prediction convert the  number to character

def decode_predictions(preds, encoder):
    preds = preds.permute(1, 0, 2)
    preds = torch.softmax(preds, 2)
    preds = torch.argmax(preds, 2)
    preds = preds.detach().cpu().numpy()
    cap_preds = []
    for j in range(preds.shape[0]):
        temp = []
        for k in preds[j, :]:
            k = k - 1
            if k == -1:
                temp.append("ยง")
            else:
                p = encoder.inverse_transform([k])[0]
                temp.append(p)
        tp = "".join(temp).replace("ยง", "")
        cap_preds.append(remove_duplicates(tp))
    return cap_preds

## **Label encoding**

In [ ]:
image_files = glob.glob(os.path.join(config.DATA_DIR,"*.png"))
targets_orig = [x.split('/')[-1][:-4] for x in image_files]
targets = [[c for c in t] for t in targets_orig]
target_flat = [c for t in targets for c in t ]

# Encoding the charecter of CAPTCHA by using the label encoding.
lbl_enc = preprocessing.LabelEncoder()
lbl_enc.fit(target_flat)
targets_enc = [lbl_enc.transform(x) for x in targets]
targets_enc = np.array(targets_enc)
targets_enc = targets_enc + 1


## **Split the data into training and testing part**

In [ ]:

train_imgs,test_imgs,train_targets,test_targets,_,test_targets_orig = model_selection.train_test_split(image_files, targets_enc, targets_orig, test_size=0.1, random_state=42)

## **Create dataset**

In [ ]:
#data set class for train data
train_dataset = dataset.ClassificationDataset(
    image_paths=train_imgs,
    targets=train_targets,
    resize=(config.IMAGE_HEIGHT, config.IMAGE_WIDTH),)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=config.BATCH_SIZE,
    num_workers=config.NUM_WORKERS,
    shuffle=True,)

#data set class for test data
test_dataset = dataset.ClassificationDataset(
    image_paths=test_imgs,
    targets=test_targets,
    resize=(config.IMAGE_HEIGHT, config.IMAGE_WIDTH),)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=config.BATCH_SIZE,
    num_workers=config.NUM_WORKERS,
    shuffle=False,)

## **Load the Captcha Model**

In [ ]:
model = CaptchaModel(num_chars=len(lbl_enc.classes_))
model.to(config.DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, factor=0.8, patience=5, verbose=True)

  

## **Train the Model**

In [ ]:
for epoch in range(70):
    train_loss = engine.train_fn(model,train_loader,optimizer)
    valid_preds, test_loss = engine.eval_fn(model, test_loader)
    valid_cap_preds = []
    for vp in valid_preds:
      current_preds = decode_predictions(vp,lbl_enc)
      valid_cap_preds.extend(current_preds)
    print(list(zip(test_targets_orig, valid_cap_preds))[6:11])
    print(f"Epoch={epoch}, Train Loss={train_loss}, Test Loss={test_loss} ") 


## Saving  the Model and other variable 

In [ ]:
import pickle
with open('gdrive/MyDrive/lbl.pickle', 'wb') as handle:
    pickle.dump(lbl_enc, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
model_save_name = 'captcha_re.pt'
path = F"/content/gdrive/My Drive/{model_save_name}"
torch.save(model, path)